In [1]:
import xml.etree.ElementTree as ET

In [2]:
rml_file_path = './data/psg_audio/rml_clean/00000995-100507.rml'

In [3]:
tree = ET.parse(rml_file_path)
root = tree.getroot()

In [4]:
print(root.tag, root.attrib)

{http://www.respironics.com/PatientStudy.xsd}PatientStudy {'StudyType': 'PSG', 'State': 'New', 'EncryptionType': '3', 'Version': '6'}


In [8]:
taglists = []

for child in root:
    taglists.append(child.tag.split('}')[1]) 

taglists

In [17]:
for child in root[1]:
    print(child.tag, child.text)

{http://www.respironics.com/PatientStudy.xsd}ConfigName LDxS_PSG_AASM-C3-C4
{http://www.respironics.com/PatientStudy.xsd}Channels 
         


In [12]:
ns = { 'study' : 'http://www.respironics.com/PatientStudy.xsd'}

for patient in root.findall('study:Patient', ns):
    gender = patient.find('study:Gender', ns)
    print(gender.text)

Female
